In [ ]:
"""Produce regression stores for cross decoding across subjects. Uses mini epochs produced in step 2D"""

import os
import mne 
import pandas as pd 
import numpy as np 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed
from collections import Counter
import seaborn as sns
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score


from sklearn.linear_model import BayesianRidge

def linear_regression_timegen(X_train, X_test, y_train_deg, t_test,t_train_window):
    "Run regression model on the sine and cosine of a given angle condition"
    "takes in a window for training times"
    y_train_sin = np.sin(np.deg2rad(y_train_deg))
    y_train_cos = np.cos(np.deg2rad(y_train_deg))
    
    # Average over training time window
    X_train_avg = X_train[:, :, t_train_window[0]:t_train_window[1]].mean(axis=2)

    # Setup pipelines
    pipe_sin = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', Ridge(alpha=1000))
    ])
    pipe_cos = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', Ridge(alpha=1000))
    ])

    # Fit on training data
    pipe_sin.fit(X_train_avg, y_train_sin)
    pipe_cos.fit(X_train_avg, y_train_cos)

    # Predict on test timepoint
    y_pred_sin = pipe_sin.predict(X_test[:, :, t_test])
    y_pred_cos = pipe_cos.predict(X_test[:, :, t_test])

    # Convert back to degrees (0–360)
    y_pred_angle = np.rad2deg(np.arctan2(y_pred_sin, y_pred_cos)) % 360
    return y_pred_angle



def run_regression_cross(epochs_train, epochs_test, target_column_train, target_column_test, train_window):
    "Create training and testing set, call function to run the regression model"
    # Extract degree targets
    y_train_deg = epochs_train.metadata[target_column_train].values.astype(float)
    y_test_deg = epochs_test.metadata[target_column_test].values.astype(float)

    # Extract full data
    X_train = epochs_train._data  # shape: (n_trials, n_channels, n_times)
    X_test = epochs_test._data

    # Run time-generalized decoding
    y_predicted = Parallel(n_jobs=8)(
        delayed(linear_regression_timegen)(
            X_train, X_test, y_train_deg,
            t_test=t,
            t_train_window=train_window
        )
        for t in range(X_test.shape[2])
    )

    # Convert to shape: (n_trials, n_times)
    y_predicted = np.array(y_predicted).T

    return y_predicted, y_test_deg


def get_train_window(peak_sample, window_size=36):
    """
    Return a window of samples centered on the peak_sample.
    """
    start = max(0, peak_sample - window_size)
    end = peak_sample + window_size
    return (start, end)

# Define the subjects
subjects = [f"S{i:02}" for i in range(1, 21)]
csv_path = "/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/LDA-16way Static/Mean/Peak_Times.csv"
peak_df = pd.read_csv(csv_path, index_col='Subject')
bids_dir = '/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
data_path = f'{bids_dir}/derivatives/preprocessed/'
output_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/errors/data'
for Subject in subjects:
    peak1_sample = int(peak_df.loc['all', 'peak1_sample'])
    train_window = get_train_window(peak1_sample, window_size=18)
    print(f"Processing {Subject}...")
    print(train_window)

    # Specify filenames
    fn_movie = f'sub-{Subject}_Dynamic_preprocessed-epo.fif'
    fn_still = f'sub-{Subject}_Still_preprocessed-epo.fif'
    fn_miniepochs = f'sub-{Subject}_miniEpochs_preprocessed-epo.fif'

    # read epochs
    epochs_movie = mne.read_epochs(data_path+fn_movie)
    epochs_still = mne.read_epochs(data_path+fn_still)
    epochs_mini = mne.read_epochs(data_path + fn_miniepochs)

    # exclude catch trials from stills & add degree column
    epochs_still.metadata['degrees_string'] = [k.split('/')[-1] for k in epochs_still.metadata['trial_type']]
    epochs_still = epochs_still[epochs_still.metadata['degrees_string'] != 'catch']
    epochs_still.metadata['degrees']=[int(i) for i in epochs_still.metadata['degrees_string']]
    epochs_still.metadata.rename(columns={'run_nr': 'run'}, inplace=True)

    epochs_mini.metadata['degrees'] = epochs_mini.metadata['degrees'].astype(int)

    target_column_train = 'degrees'
    target_column_test = 'degrees'
    epochs_train = epochs_still
    epochs_test = epochs_mini


    y_pred, y_true = run_regression_cross(epochs_train,epochs_test,target_column_train,target_column_test, train_window)
    #Save out predictions and true degree scores
    np.save(os.path.join(output_dir, f"{Subject}_predictionsEarly.npy"), y_pred)
    np.save(os.path.join(output_dir, f"{Subject}_trueEarly.npy"), y_true)
    #y_pred = np.squeeze(y_pred, axis=0)


    train_time = 0
    errors_totals = []
    #Calculate and save out errors
    for trial in range(y_pred.shape[0]):  # n_trials
        trial_errors = []
        
        #If movies are going left, error sign should be reversed
        #Negative error: prediction lagging behind where movie currently is 
        #Positive error: prediction ahead of where movie currently is
        sign_flip = -1 if 'Left' in epochs_mini.metadata['condition'].iloc[trial] else 1
        actual_angle = y_true[trial]

        for time in range(y_pred.shape[1]):  # n_timepoints (typically 60)
            predicted_angle = y_pred[trial, time]
            #print(predicted_angle)
            # Compute circular error in degrees, signed
            error = ((predicted_angle - actual_angle + 180) % 360) - 180
            #print(error)
            if trial < 2:  # Only print first two trials
                print(f"Trial {trial} Predicted: {predicted_angle}, Error: {error}")

            error *= sign_flip

            trial_errors.append(error)

        errors_totals.append(trial_errors)

    # Save as numpy array
    errors_total_array = np.array(errors_totals)
    os.makedirs(output_dir, exist_ok=True)
    #output_file = f"{output_dir}, sub-{Subject}_errorsEarly.npy"
    np.save(os.path.join(output_dir, f"{Subject}_ErrorsEarly.npy"), errors_total_array)
    #print(f"Saved errors for {Subject} to {output_file}")


Processing S01...
(371, 407)
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo-1.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
504 matching events found
No baseline correction applied
0 projection items activated
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Still_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
1104 matching events found
No baseline correction applied
0 project

In [2]:
"""Run the above again on a late window for training"""
import os
import mne 
import pandas as pd 
import numpy as np 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed
from collections import Counter
import seaborn as sns
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score


from sklearn.linear_model import BayesianRidge

def linear_regression_timegen(X_train, X_test, y_train_deg, t_test,t_train_window):
    y_train_sin = np.sin(np.deg2rad(y_train_deg))
    y_train_cos = np.cos(np.deg2rad(y_train_deg))
    
    # Average over training time window
    X_train_avg = X_train[:, :, t_train_window[0]:t_train_window[1]].mean(axis=2)

    # Setup pipelines
    pipe_sin = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', Ridge(alpha=1000))
    ])
    pipe_cos = Pipeline([
        ('scaler', StandardScaler()),
        ('regression', Ridge(alpha=1000))
    ])

    # Fit on training data
    pipe_sin.fit(X_train_avg, y_train_sin)
    pipe_cos.fit(X_train_avg, y_train_cos)

    # Predict on test timepoint
    y_pred_sin = pipe_sin.predict(X_test[:, :, t_test])
    y_pred_cos = pipe_cos.predict(X_test[:, :, t_test])

    # Convert back to degrees (0–360)
    y_pred_angle = np.rad2deg(np.arctan2(y_pred_sin, y_pred_cos)) % 360
    return y_pred_angle



def run_regression_cross(epochs_train, epochs_test, target_column_train, target_column_test, train_window):
    # Extract degree targets
    y_train_deg = epochs_train.metadata[target_column_train].values.astype(float)
    y_test_deg = epochs_test.metadata[target_column_test].values.astype(float)

    # Extract full data
    X_train = epochs_train._data  # shape: (n_trials, n_channels, n_times)
    X_test = epochs_test._data

    # Run time-generalized decoding
    y_predicted = Parallel(n_jobs=8)(
        delayed(linear_regression_timegen)(
            X_train, X_test, y_train_deg,
            t_test=t,
            t_train_window=train_window
        )
        for t in range(X_test.shape[2])
    )

    # Convert to shape: (n_trials, n_times)
    y_predicted = np.array(y_predicted).T

    return y_predicted, y_test_deg


def get_train_window(peak_sample, window_size=36):
    """
    Return a window of samples centered on the peak_sample.
    """
    start = max(0, peak_sample - window_size)
    end = peak_sample + window_size
    return (start, end)

# Define the subjects
subjects = [f"S{i:02}" for i in range(1, 21)]
csv_path = "/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/LDA-16way Static/Mean/Peak_Times.csv"
peak_df = pd.read_csv(csv_path, index_col='Subject')
# load data [NOTE @Sebastian: derivatives should be in bids_dir, i.e. "..../Data/Bids/bids_dir" to be compatible]
bids_dir = '/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
data_path = f'{bids_dir}/derivatives/preprocessed/'
output_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/errors/data'
for Subject in subjects:
    peak1_sample = int(peak_df.loc['all', 'peak1_sample'])
    train_window = get_train_window(peak1_sample, window_size=18)
    print(f"Processing {Subject}...")
    print(train_window)

    # Specify the custom filename from which you want to load the pickle file
    fn_movie = f'sub-{Subject}_Dynamic_preprocessed-epo.fif'
    fn_still = f'sub-{Subject}_Still_preprocessed-epo.fif'
    fn_miniepochs = f'sub-{Subject}_miniEpochsBaselineIncluded_preprocessed-epo.fif'

    # read epochs
    epochs_movie = mne.read_epochs(data_path+fn_movie)
    epochs_still = mne.read_epochs(data_path+fn_still)
    epochs_mini = mne.read_epochs(data_path + fn_miniepochs)

    # exclude catch trials from stills & add degree column
    epochs_still.metadata['degrees_string'] = [k.split('/')[-1] for k in epochs_still.metadata['trial_type']]
    epochs_still = epochs_still[epochs_still.metadata['degrees_string'] != 'catch']
    epochs_still.metadata['degrees']=[int(i) for i in epochs_still.metadata['degrees_string']]
    epochs_still.metadata.rename(columns={'run_nr': 'run'}, inplace=True)

    epochs_mini.metadata['degrees'] = epochs_mini.metadata['degrees'].astype(int)






    target_column_train = 'degrees'
    target_column_test = 'degrees'
    epochs_train = epochs_still
    epochs_test = epochs_mini


    y_pred, y_true = run_regression_cross(epochs_train,epochs_test,target_column_train,target_column_test, train_window)
    np.save(os.path.join(output_dir, f"{Subject}_predictionsEarlyBaselineIncluded.npy"), y_pred)
    np.save(os.path.join(output_dir, f"{Subject}_trueEarlyBaselineIncluded.npy"), y_true)
    #y_pred = np.squeeze(y_pred, axis=0)


    train_time = 0
    errors_totals = []
    errors_totals = []

    for trial in range(y_pred.shape[0]):  # n_trials
        trial_errors = []
        
        sign_flip = -1 if 'Left' in epochs_mini.metadata['condition'].iloc[trial] else 1
        actual_angle = y_true[trial]

        for time in range(y_pred.shape[1]):  # n_timepoints (typically 60)
            predicted_angle = y_pred[trial, time]
            #print(predicted_angle)
            # Compute circular error in degrees, signed
            error = ((predicted_angle - actual_angle + 180) % 360) - 180
            #print(error)
            if trial < 2:  # Only print first two trials
                print(f"Trial {trial} Predicted: {predicted_angle}, Error: {error}")

            error *= sign_flip

            trial_errors.append(error)

        errors_totals.append(trial_errors)

    # Save as numpy array
    errors_total_array = np.array(errors_totals)
    #os.makedirs(output_dir, exist_ok=True)
    #output_file = f"/{output_dir}sub-{Subject}_errorsLate.npy"
    np.save(os.path.join(output_dir, f"{Subject}_ErrorsEarlyBaselineIncluded.npy"), errors_total_array)
    #print(f"Saved errors for {Subject} to {output_file}")


Processing S01...
(371, 407)
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo-1.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
504 matching events found
No baseline correction applied
0 projection items activated
Reading /misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Still_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
1104 matching events found
No baseline correction applied
0 project